In [1]:
import purly
import webbrowser
# if you're running on mybinder.or{"children": [], "attribute"}g
# localhost does not return 127.0.0.1
from example_utils import localhost

# A Source Of Truth

The core problem that Purly solves is data synchronization between clients - in order to allow Python to control a webpage's Document Object Model (DOM) both Python and the webpage need to share their states with eachother. Purly accomplished this by running a server which acts as a "source of truth" for its clients. This server can be run in its own process via `Machine.run`, but to get started right away we'll use `Machine.daemon` which runs a subprocess.

In [2]:
purly.state.Machine().daemon()

[2018-10-02 23:57:09 -0700] [5284] [INFO] Goin' Fast @ http://127.0.0.1:8000
[2018-10-02 23:57:09 -0700] [5284] [INFO] Starting worker [5284]


To start using the Purly model server we'll need to specify the name of the model (``model_name``) our client will view and update. We'll also need to get a URL that we can use to connect to the server. There are two relevant routes.

In [3]:
model_name = "/example-model"

webpage_url = localhost('http', 8000) + '/model' + model_name + '/assets/index.html'
websocket_url = localhost('ws', 8000) + '/model' + model_name + '/stream'

print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)
print()
print('Websockets use this route to stream layout updates:')
print(websocket_url)

Get a webpage that streams layout updates via a websocket:
http://127.0.0.1:8000/model/example-model/assets/index.html

Websockets use this route to stream layout updates:
ws://127.0.0.1:8000/model/example-model/stream


# Making Layouts

In [9]:
layout = purly.Layout(websocket_url)

In [10]:
div = layout.html('div')
layout.children.append(div)
div.style.update(height='20px', width='20px', backgroundColor='red')

In [11]:
layout.sync()

Now that you've made a layout you need to sync it with the model server with `Layout.sync`

# Display Output

Since we're trying to create visual results in the browser we need to show our work. There are a couple ways to do this:


1. **In the notebook**:
    1. Display the `Layout` object in a cell.
    2. Use `purly.display.output` with a websocket route.
2. **In your browser**:
    1. Open a new window and go to `http://my-model-server/<model>`.

In [12]:
layout

In [8]:
if not webbrowser.open(webpage_url):
    print("Open up a new browser window at %s" % webpage_url)

# Realtime Updates

Because all the displays above are connected to the same model on the same server. They can all be synced in realtime!

In [9]:
div.style['backgroundColor'] = 'blue'

In [10]:
layout.sync()

Check out the webpage too. Even it got updated!

In [11]:
@div.on('Click')
def toggle():
    if div.style['backgroundColor'] == 'blue':
        div.style['backgroundColor'] = 'red'
    else:
        div.style['backgroundColor'] = 'blue'

In [ ]:
layout.serve()